In [1]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
# Load the YAML file
with open('../../conf/global.yml', 'r') as f:
    columns = yaml.safe_load(f)

# Load the YAML file
with open('../../conf/local.yml', 'r') as f:
    credentials = yaml.safe_load(f)

# Load Data

In [2]:
driver = credentials['warehouse_db']['driver']
server = credentials['warehouse_db']['server']
database = credentials['warehouse_db']['database']
trusted = credentials['warehouse_db']['trusted_connection']
user = credentials['warehouse_db']['user']
password = credentials['warehouse_db']['password']

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      f'Server={server};'
                      f'Database={database};'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [4]:
join_tables_query = '''
select *
from DIM_COMPANY dim
join FACT_BALANCE_SHEET bs on bs.ticker = dim.ticker
join FACT_CASH_FLOW_STATEMENT cfs on cfs.PK = bs.PK
join FACT_INCOME_STATEMENT fis on fis.PK = bs.PK
join FACT_PRICES fp on fp.PK = bs.PK
join FACT_RATIOS fr on fr.PK = bs.PK
'''

In [5]:
data = pd.read_sql(join_tables_query, conn)

C:\Users\germa\AppData\Local\Temp/ipykernel_23520/3119235390.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(join_tables_query, conn)


In [6]:
data.head(2)

,ticker,company_full_name,country,industry,sector,PK,ticker,Date,Cash On Hand,Receivables,...,EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,EV / FCF,P/E,P/S,P/CF
0,AAP,Advance Auto Parts,United States,Retail & Wholesale Auto Parts,Retail/Wholesale,113,AAP,2009-12-31,100.018,92.560,...,1.023673,9.153705,12.193969,832.765478,514.151,10.776512,10.776512,14.303887,0.711332,5.502686
1,AAP,Advance Auto Parts,United States,Retail & Wholesale Auto Parts,Retail/Wholesale,114,AAP,2010-12-31,59.209,124.227,...,1.353698,10.703573,13.712572,832.765478,466.809,17.182482,17.182482,16.746836,0.973014,8.654546


In [9]:
# drop duplicated columns
data = data.T.drop_duplicates().T

# Analysis

In [12]:
data.head(2)

,ticker,company_full_name,country,industry,sector,PK,Date,Cash On Hand,Receivables,Inventory,...,Enterprise Value (EV),EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,P/E,P/S,P/CF
0,AAP,Advance Auto Parts,United States,Retail & Wholesale Auto Parts,Retail/Wholesale,113,2009-12-31,100.018,92.56,1631.867,...,5540.754196,1.023673,9.153705,12.193969,832.765478,514.151,10.776512,14.303887,0.711332,5.502686
1,AAP,Advance Auto Parts,United States,Retail & Wholesale Auto Parts,Retail/Wholesale,114,2010-12-31,59.209,124.227,1863.87,...,8020.937383,1.353698,10.703573,13.712572,832.765478,466.809,17.182482,16.746836,0.973014,8.654546


In [14]:
data['Date'] = pd.to_datetime(data['Date'], format="%Y/%m/%d")

In [15]:
columns

{'column_id': ['PK', 'ticker', 'Date'],
 'columns_income_statement': ['Revenue',
  'Cost Of Goods Sold',
  'Gross Profit',
  'Research And Development Expenses',
  'SG&A Expenses',
  'Other Operating Income Or Expenses',
  'Operating Expenses',
  'Operating Income',
  'Total Non-Operating Income/Expense',
  'Pre-Tax Income',
  'Income Taxes',
  'Income After Taxes',
  'Other Income',
  'Income From Continuous Operations',
  'Income From Discontinued Operations',
  'Net Income',
  'EBITDA',
  'EBIT',
  'Basic Shares Outstanding',
  'Shares Outstanding',
  'Basic EPS',
  'EPS - Earnings Per Share'],
 'columns_balance_sheet': ['Cash On Hand',
  'Receivables',
  'Inventory',
  'Pre-Paid Expenses',
  'Other Current Assets',
  'Total Current Assets',
  'Property, Plant, And Equipment',
  'Long-Term Investments',
  'Goodwill And Intangible Assets',
  'Other Long-Term Assets',
  'Total Long-Term Assets',
  'Total Assets',
  'Total Current Liabilities',
  'Long Term Debt',
  'Other Non-Current 